In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import cross_val_score

from sklearn.impute import KNNImputer
from pandas.api.types import CategoricalDtype
from geopy.geocoders import Nominatim
import re

import pandas as pd

from scipy import stats

In [2]:
from my_pipelines import *
from kacper_pipelines import *
from pipelines_miki import *

In [3]:
FILE_PATH = './weatherAUS.csv'

In [4]:
australia_rain = pd.read_csv(FILE_PATH)
australia_rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
# pipe = Pipeline([("drop", DropColumns("Location"))])
# pipe.fit_transform(australia_rain)

In [16]:
class WholeRainClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, clf_class = RandomForestClassifier,
                 columns_na_threshold = 12,
                 temp_daily_difference = True,
                 wind_daily_difference = True,
                 pressure_daily_difference = True,
                 humidity_daily_difference = True,
                 latitude_longnitude = True,
                 latitude_longitude_polynomial = 4,
                 imputation = "mean",
                 wind_transformation = "wind_to_degrees",
                 date_features = True
                ):
        self.clf_class = clf_class
        self.columns_na_threshold = 12
        self.temp_daily_difference = temp_daily_difference
        self.wind_daily_difference = wind_daily_difference
        self.pressure_daily_difference = pressure_daily_difference
        self.humidity_daily_difference = humidity_daily_difference
        self.latitude_longnitude = latitude_longnitude
        self.latitude_longitude_polynomial = latitude_longitude_polynomial
        self.imputation = imputation
        self.wind_transformation = wind_transformation
        self.date_features = date_features
    def fit(self, X, y):
        pipeline_candidates = []
        
        if self.latitude_longnitude:
            pipeline_candidates.append(("latitude_longnitude", MapLocation(X, normalize=False)))
            
        if self.wind_transformation == "wind_to_degrees":
            pipeline_candidates.append(("wind_to_degrees",WindToDegrees()))
        elif self.wind_transformation == "wind_to_binary":
            raise Exception("not implemented")
        else:
            pipeline_candidates.append(("drop_wind", DropColumns(["WindGustDir", "WindDir9am", "WindDir3pm"])))
        #remove bad columns
        pipeline_candidates.append(("Drop columns with NAs", DropColumnsAbovePercentNA(12)))
        
        #normalization
        pipeline_candidates.append(("normalization", NormalizeContinuousFeatures(MinMaxScaler())))
        
        if self.imputation == "mean":
            #IMupte
            pipeline_candidates.append(("NA Mean Imputer", MeanNANImputer()))
                
        #custom features
        if self.temp_daily_difference:
            pipeline_candidates.append(("temp_daily_difference", MaxMinTempDifference()))
        if self.wind_daily_difference:
            pipeline_candidates.append(("wind_daily_difference", WindDailyDifference()))
        if self.pressure_daily_difference:
            pipeline_candidates.append(("pressure_daily_difference", PressureDailyDifference()))
        if self.humidity_daily_difference:
            pipeline_candidates.append(("humidity_daily_difference", HumidityDailyDifference()))
        if self.latitude_longnitude and self.latitude_longitude_polynomial:
            pipeline_candidates.append(("latitude_longnitude_polynomial", PolynomialSubset(['longitude', 'latitude'], self.latitude_longitude_polynomial)))
        
        if self.date_features:
            pipeline_candidates.append(("Date", FeaturesFromDate(True)))
        else:
            pipeline_candidates.append(("Drop Date", DropColumns(["Date"])))
        
        
        pipeline_candidates.append(("Drop Rest", DropColumns(["Location", 'Temp3pm','Temp9am','Humidity9am'])))
        #add classifier
        pipeline_candidates.append(("classifier", self.clf_class(n_jobs=-1)))
        
        self.pipeline = Pipeline(pipeline_candidates)
        self.pipeline.fit(X,y)
        return self
    def predict(self, X):
        return self.pipeline.predict(X)

In [17]:
class EachCityClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.classifiers = dict()# dict of pipelines
    def fit(self, australia_rain, y):
        main_pipeline_candidates = []
        #drop without RainTommorrow
        main_pipeline_candidates.append(("drop_without_class", DropColumnsWithNAs("RainTomorrow")))
        
        australia_rain_by_city = {k:v for k, v in australia_rain.groupby('Location')}
        
        pass
    def predict(self, X):
        pass

In [25]:
australia_rain = pd.read_csv(FILE_PATH)
print(australia_rain.shape)
australia_rain = DropRowsWithNAInColums(["RainTomorrow"]).transform(australia_rain)
australia_rain = DropRowsWithMoreThanXNA(10).transform(australia_rain)
australia_rain = RainToNumerical().transform(australia_rain)
australia_rain = MeanNANImputer().fit_transform(australia_rain)
australia_rain = RemoveOutliers().fit_transform(australia_rain)

(145460, 23)
Removed: 4560


In [19]:
australia_rain

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.600000,5.468002,7.624739,W,44.000000,W,...,71.0,22.0,1007.7,1007.1,8.000000,4.503452,16.9,21.8,0.0,0
1,2008-12-02,Albury,7.4,25.1,0.000000,5.468002,7.624739,WNW,44.000000,NNW,...,44.0,25.0,1010.6,1007.8,4.440081,4.503452,17.2,24.3,0.0,0
2,2008-12-03,Albury,12.9,25.7,0.000000,5.468002,7.624739,WSW,46.000000,W,...,38.0,30.0,1007.6,1008.7,4.440081,2.000000,21.0,23.2,0.0,0
3,2008-12-04,Albury,9.2,28.0,0.000000,5.468002,7.624739,NE,24.000000,SE,...,45.0,16.0,1017.6,1012.8,4.440081,4.503452,18.1,26.5,0.0,0
4,2008-12-05,Albury,17.5,32.3,1.000000,5.468002,7.624739,W,41.000000,ENE,...,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144926,2016-01-09,Uluru,28.8,38.4,2.341913,5.468002,7.624739,NW,48.000000,ENE,...,17.0,13.0,1013.7,1011.0,8.000000,6.000000,32.8,36.8,NaN,0
144951,2016-02-03,Uluru,20.6,32.6,2.341913,5.468002,7.624739,S,39.000000,SE,...,50.0,32.0,1011.5,1008.7,4.440081,4.503452,22.6,30.0,NaN,0
144956,2016-02-08,Uluru,23.8,38.3,2.341913,5.468002,7.624739,NaN,39.979108,ESE,...,27.0,16.0,1015.2,1010.8,4.440081,4.503452,30.6,36.1,NaN,0
145093,2016-06-24,Uluru,3.2,17.9,2.341913,5.468002,7.624739,E,44.000000,SE,...,83.0,39.0,1022.4,1019.0,4.440081,4.503452,8.8,17.2,NaN,0


In [20]:
y = australia_rain["RainTomorrow"].astype(int)
X = australia_rain.drop("RainTomorrow", axis =1)
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

(135838, 22)


In [21]:
clf = WholeRainClassifier(date_features = False)

In [22]:
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)

In [23]:
score

0.8643502159403219

In [24]:
scores = cross_val_score(WholeRainClassifier(date_features = False), X, y, cv=10, scoring="f1")
scores

array([0.50751969, 0.47093538, 0.52605304, 0.56170045, 0.59108808,
       0.55895197, 0.56540256, 0.55819866, 0.51892088, 0.45465656])

In [26]:
sum(scores)/len(scores)

0.5313427267402357

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [ ]:
columns_na_threshold = 12,
                 temp_daily_difference = True,
                 wind_daily_difference = True,
                 pressure_daily_difference = True,
                 humidity_daily_difference = True,
                 latitude_longnitude = True,
                 latitude_longitude_polynomial = 4,
                 imputation = "mean",
                 wind_transformation = "wind_to_degrees",
                 date_features = True
                    XGBClassifier(objective='binary:logistic')

In [43]:
gscv = GridSearchCV(WholeRainClassifier(),
             param_grid={'clf_class': [RandomForestClassifier,MLPClassifier]},
            n_jobs=-1, cv=2)

In [44]:
gscv.fit(X_train,y_train)

TypeError: get_params() missing 1 required positional argument: 'self'

In [45]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(WholeRainClassifier())

TypeError: get_params() missing 1 required positional argument: 'self'